In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [2]:
%run "{project_path}\llm_custom_apps\llm_apps\oh_sheet_its_spark\utils.ipynb"

In [3]:
# Initialize an empty list to maintain chat history & message history
conversation_history = []
msg_history = []
formatted_history = ""  # Variable to store the formatted chat history
inf_mdl_nm = "Qwen/Qwen2.5-Coder-32B-Instruct"
file_uploaded = False  # Flag to track if a file is uploaded

# Function to handle file upload
def handle_file_upload(file):
    global file_uploaded, conversation_history, msg_history, formatted_history
    if file:
        # Reset histories on new file upload
        conversation_history = []
        msg_history = []
        formatted_history = ""  # Reset formatted history as well

        file_uploaded = True  # Set the flag to True when a file is uploaded

        # Extract the file name
        file_name = file.name

        initial_msg = gen_initial_msg(file_name)
        cd_resp = generate_responses_from_inf(hf_token, initial_msg, inf_mdl_nm, 5000)

        # Add the "File Loaded" message to msg_history only
        msg_history.append(initial_msg[0])
        msg_history.append(initial_msg[1])
        
        # Add the bot's response with the file name to both conversation_history and msg_history
        conversation_history.append(("Bot", "\n\n" + cd_resp))
        msg_history.append({'role': "assistant", 'content': cd_resp})

        # Format the initial bot message and append to formatted history
        formatted_history = format_message("Bot", "\n\n" + cd_resp)

        # Generate the formatted chat history for display
        return "File uploaded successfully. You can now start chatting!", formatted_history
    else:
        file_uploaded = False
        return "Please upload a valid Excel file.", ""

# Function to format each message with proper styling
def format_message(speaker, message):
    if speaker == "User":
        return f'<div style="background-color: #FFEBEE; padding: 10px; border-radius: 10px; margin-bottom: 10px; text-align: right; color: #6a1b9a;"><strong>{speaker}:</strong> {message}</div>'
    else:
        # Bot messages now have the word "Bot:" styled in #cd7337 color
        return f'<div style="background-color: #e0f7fa; padding: 10px; border-radius: 10px; margin-bottom: 10px; text-align: left; color: #000;"><strong style="color: #cd7337;">{speaker}:</strong> {message}</div>'

# Function to handle chat history and generate responses
def chatbot_history(user_input):
    global conversation_history  # Use the global conversation_history list
    global msg_history
    global formatted_history  # Use the global formatted_history

    # Append user input to the conversation history
    conversation_history.append(("User", user_input))
    msg_history.append({'role': "user", 'content': str(user_input)})

    # Generate a response (for testing, reversing user input)
    bot_response = generate_responses_from_inf(hf_token, msg_history, inf_mdl_nm, 5000)
    msg_history.append({'role': "assistant", 'content': bot_response})

    print("msg_history: ", msg_history)

    # Append bot response to conversation history
    conversation_history.append(("Bot", bot_response))

    # Format the current user and bot message
    current_conversation = format_message("User", user_input) + format_message("Bot", "\n\n" +bot_response)

    # Combine previous formatted history with the new formatted message
    formatted_history += current_conversation

    # Return the updated formatted history
    return gr.update(value=formatted_history), ""  # Return the current formatted history

# Function to enable chat input and submit button after file upload
def enable_chat_components(file_status):
    if "successfully" in file_status.lower():
        return gr.update(interactive=True), gr.update(interactive=True)
    else:
        return gr.update(interactive=False), gr.update(interactive=False)

# Create Gradio interface using Blocks with Soft theme
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # Add a title with proper font
    gr.HTML(""" 
        <div style="font-size: 2.5em; color: #4CAF50; font-weight: bold; text-align: center;">
            Oh Sheet!!!!! It's Spark
        </div>
    """)

    # Layout with chat and input section
    with gr.Row():
        # Message section on the left (30% width)
        with gr.Column(scale=1, min_width=30):
            # File uploader at the top
            file_upload = gr.File(label="Upload an Excel File", file_types=[".xls", ".xlsx"], interactive=True)
            file_status = gr.Textbox(label="", interactive=False, lines=1, value="Please upload an Excel file to begin.", show_label=False)

            # Chatbox and submit button
            input_text = gr.Textbox(label="Enter your message", lines=1, interactive=False)  # Initially disabled
            submit_button = gr.Button("Submit", variant="primary", interactive=False)  # Initially disabled

        # Chatbot section on the right (70% width)
        with gr.Column(scale=2, min_width=70):
            output_text = gr.Markdown(label="Chatbot History", elem_id="chat_history")  # Use Markdown for rendering content

    # Ensure file upload enables the chatbox
    file_upload.change(fn=handle_file_upload, inputs=file_upload, outputs=[file_status, output_text])
    file_status.change(fn=enable_chat_components, inputs=file_status, outputs=[input_text, submit_button])

    # Ensure Enter key is mapped to submit functionality
    input_text.submit(fn=chatbot_history, inputs=input_text, outputs=[output_text, input_text])
    submit_button.click(fn=chatbot_history, inputs=input_text, outputs=[output_text, input_text])

# Launch the app
demo.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


msg_history:  [{'role': 'system', 'content': "\nYou are a helpful assistant to a Data Scientist who is working on a project to transform Excel Spreadsheets to PySpark Dataframes. \nThe Data Scientist has provided you with a dictionary of dictionaries which contains details about Excel spreadsheets.\nEach dictionary has only one key which is the name of the Sheet and the value is a dictionary with multiple keys.\nEach key is the name of the Column Header in the Excel Spreadsheet. The value is again a dictionary with two keys.\nThe first key is 'ColumnID' with the value being the associated ColumnID in Excel i.e. A, B, C, or etc.\nThe second key is 'ColumnValue' with the value being the associated formula for generating the column or the hardcoded value in the absence of the formula.\nThe dictionary looks something like:\n\n{{'SheetNum1': {'X': {'ColumnID': 'A', 'ColumnValue': 11},\n   'Y': {'ColumnID': 'B', 'ColumnValue': 2020},\n   'Z': {'ColumnID': 'C', 'ColumnValue': 15789},\n   'P':